# Final Project for Data Analytics

## Alyssa June Pacleb, Ravi Goparaju, Adeena Amersi, Ryan Chiapoco

In [4]:
import warnings
warnings.simplefilter("ignore")
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt
import sklearn.preprocessing as preproc
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, GridSearchCV
import time
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.naive_bayes import GaussianNB

%matplotlib inline

# Science Giving Column Meanings and Types

Header | Definition | Type
---|---------|-------|
`cmte_nm` | Committee name | object
`cmte_id` | Committee identification, assigned | object
`cmte_tp` | Committee type. [List of committee type codes](https://www.fec.gov/campaign-finance-data/committee-type-code-descriptions/)  | object
<font size="5">**`cmte_pty`**</font>  | **Committee party. [List of party codes](https://www.fec.gov/campaign-finance-data/party-code-descriptions/)** | **object**
`cand_name` | Candidate name | object
`cand_pty_affiliation` | Political party affiliation reported by the candidate | object
`cand_office_st` | Candidate state | object
<font size="5">**`cand_office`**</font>  | Candidate office. H = House, P = President, S = Senate | object
`cand_office_district` | Candidate district | float64
`cand_status` | Candidate status. C = Statutory candidate, F = Statutory candidate for future election, N = Not yet a statutory candidate, P = Statutory candidate in prior cycle | object
`rpt_tp` | Report type. [Report type codes](https://www.fec.gov/campaign-finance-data/report-type-code-descriptions/) | object
<font size="5">**`transaction_pgi`**</font> | The code for which the contribution was made. EYYYY (election plus election year). P = Primary, G = General, O = Other, C = Convention, R = Runoff, S = Special, E = Recount | object
`transaction_tp` | Transaction type. [Type codes](https://www.fec.gov/campaign-finance-data/transaction-type-code-descriptions/) | object
`entity_tp` | Entity type. Only valid for electronic filings received after April 2002. CAN = Candidate, CCM = Candidate Committee, COM = Committee, IND = Individual (a person), ORG = Organization (not a committee and not a person), PAC = Political Action Committee, PTY = Party Organization | object
`cleaned_name` | Contributor/lender/transfer name | object
`city` | City/town | object
<font size="5">**`state`**</font> | State | object
`zip_code` | Zip code | object
`employer` | Employer | object
<font size="5">**`cleaned_occupation`**</font>  | Occupation | object
`classification` | Classification of occupation | object
`transaction_dt` | Transaction date (MMDDYYYY) | float64
<font size="5">**`cycle`**</font> | Election cycle | float64
`transaction_amt` | Transaction amount | float64
<font size="5">**`2016_dollars`**</font>  | Transation amount adjusted for inflation | float64
`other_id` | Other identification number. For contributions from individuals this column is null. For contributions from candidates or other committees this column will contain that contributor's FEC ID. | object
`tran_id` | Transaction ID | object
`file_num` | A unique identifier associated with each itemization or transaction appearing in an FEC electronic file. Only valid for electronic filings. | float64
`memo_cd` | 'X' indicates that the amount is not to be included in the itemization total. | object
`memo_text` | A description of the activity. | object
`sub_id` | FEC record number | float64


In [108]:
# Don't Run this numerous times for the sake of time
url = 'https://media.githubusercontent.com/media/fivethirtyeight/data/master/science-giving/science_federal_giving.csv'
df = pd.read_csv(url, error_bad_lines=False)

In [109]:
print(df.shape)
df.head()

(881793, 31)


,cmte_nm,cmte_id,cmte_tp,cmte_pty,cand_name,cand_pty_affiliation,cand_office_st,cand_office,cand_office_district,cand_status,...,transaction_dt,cycle,transaction_amt,2016_dollars,other_id,tran_id,file_num,memo_cd,memo_text,sub_id
0,AMERICAN WAY - DURANT 2012; THE,C00501452,S,REP,"DURANT, CLARK",REP,MI,S,0.0,C,...,3302012.0,2012.0,2500.0,2612.50,NaN,SA0108160812336,845273.0,NaN,NaN,1.010000e+18
1,A LOT OF PEOPLE FOR DAVE OBEY,C00017830,H,DEM,"OBEY, DAVID R",DEM,WI,H,7.0,C,...,4182007.0,2008.0,250.0,292.50,NaN,C34559,294636.0,NaN,NaN,4.070000e+18
2,A WHOLE LOT OF PEOPLE FOR GRIJALVA CONGRESSION...,C00374058,H,DEM,"GRIJALVA, RAUL M",DEM,AZ,H,3.0,C,...,6092016.0,2016.0,50.0,50.00,C00401224,VSH8VCDY4S4,1096417.0,NaN,* EARMARKED CONTRIBUTION: SEE BELOW,4.080000e+18
3,A WHOLE LOT OF PEOPLE FOR GRIJALVA CONGRESSION...,C00374058,H,DEM,"GRIJALVA, RAUL M",DEM,AZ,H,3.0,C,...,8192012.0,2012.0,2500.0,2612.50,NaN,C6548162,863428.0,NaN,NaN,4.030000e+18
4,A WHOLE LOT OF PEOPLE FOR GRIJALVA CONGRESSION...,C00374058,H,DEM,"GRIJALVA, RAUL M",DEM,AZ,H,7.0,C,...,10292010.0,2010.0,250.0,273.25,NaN,C5156783,863417.0,NaN,NaN,4.030000e+18


# Data preprocessing & dimensionality reduction

In [110]:
#df['2016_dollars'] = pd.to_numeric(df['2016_dollars'])
#donation_mns = df['2016_dollars'].mean()
#df['2016_dollars'].fillna(donation_mns, inplace = True)

# print(df.isna().sum()) # Clearly there is a bit of an issue here

In [111]:
# These are realistically the only features we care about
df = df.drop(columns=['cand_pty_affiliation', 'cand_name', 'cand_office_st', 'cleaned_name','employer','cmte_nm', 'cmte_id' , 'cmte_tp', 'cand_office_district', 'transaction_amt', 'cand_status', 'rpt_tp', 'transaction_tp', 'tran_id', 'file_num', 'memo_cd', 'memo_text', 'sub_id', 'other_id', 'zip_code', "cand_name", 'entity_tp', 'city','transaction_dt'])

# I only want to look at the general presidential election donations
# data = (new[new['transaction_pgi'] == 'G']) 
# data = (data[data['cand_office'] == 'P'])
print(df.isna().sum()) 
# print(new.shape)
df.head()

cmte_pty             411796
cand_office          497992
transaction_pgi      451317
state                412250
cleanedoccupation    411590
classification       411590
cycle                411597
2016_dollars         411619
dtype: int64


,cmte_pty,cand_office,transaction_pgi,state,cleanedoccupation,classification,cycle,2016_dollars
0,REP,S,P,MI,ENGINEER,Engineer,2012.0,2612.50
1,DEM,H,P,AZ,CIVIL ENGINEER,Engineer,2008.0,292.50
2,DEM,H,P,MA,ENGINEER (SOFTWARE),Engineer,2016.0,50.00
3,DEM,H,P,TX,ENGINEER (SOFTWARE),Engineer,2012.0,2612.50
4,DEM,H,G,NaN,(CHEMIST),Scientist,2010.0,273.25


In [112]:
# Replace the column names to make the table more readable
df.rename(columns = {'cmte_pty': 'committee_party', 'cand_office': 'candidate_office'}, inplace = True)
df.head()
# test = new.groupby('transaction_pgi')
# test.first()

,committee_party,candidate_office,transaction_pgi,state,cleanedoccupation,classification,cycle,2016_dollars
0,REP,S,P,MI,ENGINEER,Engineer,2012.0,2612.50
1,DEM,H,P,AZ,CIVIL ENGINEER,Engineer,2008.0,292.50
2,DEM,H,P,MA,ENGINEER (SOFTWARE),Engineer,2016.0,50.00
3,DEM,H,P,TX,ENGINEER (SOFTWARE),Engineer,2012.0,2612.50
4,DEM,H,G,NaN,(CHEMIST),Scientist,2010.0,273.25


In [113]:
# # NOW do we take care of the missing state values or do we just delete those rows?
# # delete the nan rows. 
# copy = data.copy()
# copy = copy.dropna()
# print(copy.shape)
# print(copy.isna().sum()) 
# copy.head()

df.dropna(inplace = True)
print(df.isna().sum())

committee_party      0
candidate_office     0
transaction_pgi      0
state                0
cleanedoccupation    0
classification       0
cycle                0
2016_dollars         0
dtype: int64


In [159]:
# Data Exploration
print(df.isna().sum())

committee_party      0
candidate_office     0
transaction_pgi      0
state                0
cleanedoccupation    0
classification       0
cycle                0
2016_dollars         0
dtype: int64


## Feature Engineering

In [193]:
new = df.copy()
# We want to drop this when we perform One-hot encoding later, since there are 50 unique states that will add 50 columns,
# and cleaned occupation 9 - Actually I am now using Ordinal Encoder instead, so state can stay
new = new.drop(columns = {'state','cleanedoccupation'}) 

# Standardize the label of the Democratic party
new['committee_party'].replace(to_replace = 'dem', value = 'DEM', inplace = True)
new['committee_party'].replace(to_replace = 'Dem', value = 'DEM', inplace = True)

# Standardize the label of the Republican party
new['committee_party'].replace(to_replace = 'rep', value = 'REP', inplace = True)
new['committee_party'].replace(to_replace = 'Rep', value = 'REP', inplace = True)

# Keep only Dem, Rep, and Ind parties in the committee_party column:
new = new[(new['committee_party'] == 'DEM') | (new['committee_party'] == 'REP') | (new['committee_party'] == 'IND') ]

# Keep only General and Primary election values in the transaction_pgi column
new = new[(new['transaction_pgi'] == 'G') | (new['transaction_pgi'] == 'P')]

# new.head()

# Now it's time to perform One-Hot Encoding to turn the categorical variables into numerical ones.
# But first we should change 'cycle' and '2016_dollars' to integer type just in case:
new['cycle'] = new['cycle'].astype(int)
new['2016_dollars'] = new['2016_dollars'].astype(int)

# Now separate the columns that are already int type from the string types, encode the string types, and reattach them
# back into one DataFrame:
temp1 = new['cycle'].values
temp2 = new['2016_dollars'].values
new = new.drop(columns = {'cycle', '2016_dollars'})

# One-Hot-Encoding:
enc = preproc.OrdinalEncoder()
transformed = enc.fit_transform(new)
new = pd.DataFrame(transformed)

# Reattach columns from before
# new = pd.concat([new, temp], axis=1, sort=False) Somehow this is adding NaN's to the data
new['cycle'] = temp1
new['2016_dollars'] = temp2
# new.shape
new.rename(columns = {0:'committee_party', 1:'candidate_office', 2:'transaction_pgi', 3:'classification'}, inplace = True)

# new.rename(columns = {0:'committee_DEM', 1:'committee_IND', 2:'committee_REP',
#                                           3:'office_H', 4:'office_P', 5:'office_S',
#                                           6:'trans_pgi_G',7:'trans_pgi_P',
#                                           8:'class_Engineer', 9:'class_Math-Stat', 10:'class_scientist'}, 
#                                            inplace = True)

new.head()
# print(new.isna().sum())
# new.groupby('committee_party').first()

,committee_party,candidate_office,transaction_pgi,classification,cycle,2016_dollars
0,2.0,2.0,1.0,0.0,2012,2612
1,0.0,0.0,1.0,0.0,2008,292
2,0.0,0.0,1.0,0.0,2016,50
3,0.0,0.0,1.0,0.0,2012,2612
4,0.0,0.0,0.0,2.0,2010,546


## Data Analysis - Decision Tree and Naive Bayes

In [194]:
class_label = new['committee_party']
features = new.drop(columns = ['committee_party'])

# Naive Bayes classifier:
nbc = GaussianNB()
nb_score = cross_val_score(nbc, features, class_label, cv = 10).mean()
print(nb_score)

0.7487093241380476


In [201]:
# Decision Tree Classifier - We want to do K-fold Cross-Validation using Grid Search
# X_train, X_test, Y_train, Y_test = train_test_split(features, class_label, test_size = 0.2, train_size = 0.8)
decision_tree = DecisionTreeClassifier(criterion = 'entropy')
grid_search = GridSearchCV(decision_tree, param_grid = {'max_depth': [5,10,15,20], 'max_features': [2,3,5],
                'min_samples_leaf': [5,10,15,20]}, cv=5, scoring = 'accuracy')
grid_search.fit(features, class_label)
# print(grid_search.best_params_)

print(cross_val_score(grid_search, features, class_label, cv = 10).mean())


# X_train, X_test, Y_train, Y_test = train_test_split(features, class_label, test_size = 0.2, train_size = 0.8)
# decision_tree = decision_tree.fit(X_train, Y_train)
# # decision_tree.tree_.max_depth

# pred_vals = decision_tree.predict(Y_test, check_input = True)
# accuracy_score(pred_vals, Y_test)

0.7797726860275871
